In [ ]:
import _09_utility as util

dataset = util.load_large_csv('spx_newest.csv')
fssr = util.fn_all_subsets(True,0,True)
fssd = util.fnsubset_to_indexdictlist(dataset.columns,fssr)
fss = []
for d in fssd:
    fss.append(list(d.keys()))
print(fss)
dataset = dataset.values
direction = 1
if(direction==1):
	arr_ext = dataset[:,1]
else:
    arr_ext = dataset[:,0]
arr_close = dataset[:,2]

In [ ]:
log_normalize = True
lag_allowance = 30
generations = 10
hold_for = 20
log_normalize = True
lag_allowance = 30
generations = 10
num_eras = 1
criteria = 'r2'
elite_criteria = 1
num_parents = 10
rep_mode = 'exponential'
part_mproba = 0.000
ptrn_mproba = 0.000
using_batches = False
recomp_loaded_batches = []

In [ ]:
import warnings
warnings.simplefilter("error", category=RuntimeWarning)

import _00_gene as _0
import _01_initialization as _1
import _02_evaluation as _2
import _05_reproduction as _5
import _10_era as _10

In [ ]:
import _00_gene as _0
load_genes = _0.load_custom_genes(
	fss=fss,lag_allowance=lag_allowance,gene_pattern_sets=[[#0
        (412, 10, 'gt', 412, 14),(454, 8, 'gt', 464, 24)],[#1
		(45, 1, 'gt', 46, 7),(6, 24, 'gt', 6, 16)],[#2
		(199, 16, 'lt', 164, 15),(421, 19, 'lt', 410, 22)],[#3
		(412, 12, 'gt', 408, 19),(454, 8, 'gt', 476, 10)],[#4
		(411, 23, 'gt', 419, 22),(204, 25, 'lt', 154, 21)],[#5
		(411, 4, 'lt', 444, 13),(514, 7, 'lt', 494, 8)],[#6
		(423, 22, 'lt', 423, 14),(428, 5, 'lt', 428, 20)],[#7
		(212, 10, 'lt', 207, 18),(77, 13, 'lt', 68, 4)],[#8
		(440, 27, 'gt', 437, 0),(423, 12, 'lt', 423, 6)],[#9
		(438, 27, 'lt', 435, 17),(403, 6, 'gt', 401, 0)],[#10
		(466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29)],[#11        
		(514, 2, 'lt', 491, 16),(428, 23, 'gt', 428, 14),(417, 26, 'gt', 419, 17)],[#12
        (428, 5, 'lt', 428, 20),(466, 20, 'gt', 470, 15)],[#13
        (440, 27, 'gt', 437, 0),(411, 23, 'gt', 419, 22),(466, 20, 'gt', 470, 15)],[#14
        (436, 10, 'lt', 440, 29),(428, 5, 'lt', 428, 20),(412, 10, 'gt', 412, 14)],[#15
        (412, 10, 'gt', 412, 14),(454, 8, 'gt', 464, 24),(461, 13, 'lt', 470, 22)],[#16
        (466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29),(3, 16, 'lt', 440, 29)],[#17
        (466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29),(436, 15, 'gt', 439, 0)],[#18
        (466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29),(3, 17, 'lt', 3, 3)],[#19
        (466, 20, 'gt', 470, 15),(436, 10, 'lt', 440, 29),(3, 10, 'lt', 3, 0)],[#20
		(400, 15, 'lt', 405, 15),(339, 26, 'gt', 332, 7)],[#21
        (522, 28, 'lt', 522, 16),(192, 19, 'lt', 183, 30)],[#22
        (176, 30, 'gt', 186, 16),(543, 29, 'gt', 536, 2)]
        ])

In [ ]:
arr_returns, arr_kratio = _1.collect_parallel_metrics(
    direction=direction,
	arr_close=arr_close,
	arr_ext=arr_ext,
	hold_for=hold_for,
	lag_allow=lag_allowance,
	log_normalize=log_normalize
)

In [ ]:
era_args = {
		'generations'	:	1,
		'dataset'		:	dataset,
		'fss'			:	fss,
		'criteria'	:	criteria,
        'with_array'	:	False,
		'log_normalize'	:	log_normalize,
		'hold_for'	:	hold_for,
		'lag_allowance'	:	lag_allowance,
		'arr_returns'	:	arr_returns,
		'arr_kratio'	:	arr_kratio,
		'elite_criteria' : 1,
		'num_parents' : 4,
		'rep_mode' : 'exponential',
		'part_mproba' : 0.000,
		'ptrn_mproba' : 0.000,
		'use_strict_filter'	:	True,
		'strict_filter_kwargs'	:	{
			'avg_return':(3.50 / 25000),
			'tot_return':0.25,
			'profit_factor':1.25,
			'kelsch_ratio':1.0,
			'entry_frequency':0.005,
			'r2':0.75
		}}

In [ ]:
new_population = _1.generate_initial_population(
	sample_size=1000,
	pattern_size=2,
	feat_idx_pool=fss,
	lag_allowance=lag_allowance,
	skew_lag_prob=False
)

In [ ]:
import _05_reproduction as _5

recomp_loaded = _5.expansive_recomposition(
    population=load_genes,
    num_patterns=2
)

In [ ]:
import _05_reproduction as _5

evo_pop = _5.evolutionary_branch(
	gene=load_genes[21],
    branch_size=250
)

In [ ]:
import _05_reproduction as _5
from importlib import reload
reload(_5)
reload(_0)

addv_pop = _5.additive_branch(
    gene=load_genes[21]
    ,branch_size=250
    ,fss=fss
)

In [ ]:
import _11_eon as _11

endeon = _11.autobatch_eon(
    eon_num=0,
    population=addv_pop,
    criteria=criteria,
    filter_tightening={},
    era_kwargs=era_args,
    spec_batch_size=100
)

In [ ]:
import copy
endeon_backup = copy.deepcopy(endeon)

In [ ]:
endeon = copy.deepcopy(endeon_backup)
endeon_backup = copy.deepcopy(endeon)
len(endeon_backup)

In [ ]:
import _10_era as _10

criteria = 'r2'

endtime = _10.era(
    eon_num=0, 
	era_num	=	0,
	new_population	=	endeon,
	generations	=	1,
	dataset		=	dataset,
	fss			=	fss,
	criteria	=	criteria,
    with_array=True,
	log_normalize	=	log_normalize,
	hold_for	=	hold_for,
	lag_allowance	=	lag_allowance,
	arr_returns	=	arr_returns,
	arr_kratio	=	arr_kratio,
	elite_criteria = 1,
	num_parents = 4,
	rep_mode = 'exponential',
	part_mproba = 0.000,
	ptrn_mproba = 0.000,
	use_strict_filter	=	True,
	strict_filter_kwargs	=	{
		'avg_return':(3.5 / 25000),
		'tot_return':0.5,
		'profit_factor':1.25,
		'kelsch_ratio':2.0,
        #'mkr':0.9,
		'entry_frequency':0.005,
           'r2':0.8
}
)

In [ ]:
#reload(_2)
print(_2.show_best_gene_patterns(endtime,criteria,fssd))
_2.show_returns(endtime[0]._array_returns, arr_close=arr_close, gene_kwargs={"population":endtime,"criteria":criteria,"fss":fssd})

In [ ]:
'''new = _2.show_combined_performance(
    population=load_genes,
    arr_close=arr_close,
    arr_low=arr_low,
    arr_kratio=arr_kratio,
    arr_returns=arr_returns,
    data=dataset,
    hold_for=hold_for,
    lag_allow=lag_allowance,
    specific_data=None,
    log_normalize=log_normalize,
    criteria=criteria,
    fss=fssd
)'''